# Libraries

In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup

# Web Scrapping

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.mohfw.gov.in/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[-1]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
df_bs.drop('S. No.', axis=1, inplace=True)

# there are 36 states+UT in India
df_bs.head(36)

,Name of State / UT,Total Confirmed cases*,Cured/Discharged/Migrated,Deaths**
0,Andaman and Nicobar Islands,33,33,0
1,Andhra Pradesh,2355,1353,49
2,Arunachal Pradesh,1,1,0
3,Assam,92,41,2
4,Bihar,1179,453,7
5,Chandigarh,191,51,3
6,Chhattisgarh,67,56,0
7,Dadar Nagar Haveli,1,0,0
8,Delhi,9333,3926,129
9,Goa,17,7,0


# Data Cleaning

In [6]:
# date-time information
# =====================

# today's date
now  = datetime.now()
# format date to month-day-year
df_bs['Date'] = now.strftime("%m/%d/%Y") 

# add 'Date' column to dataframe
df_bs['Date'] = pd.to_datetime(df_bs['Date'], format='%m/%d/%Y')

# df_bs.head(36)

In [7]:
# remove extra characters from 'Name of State/UT' column
df_bs['Name of State / UT'] = df_bs['Name of State / UT'].str.replace('#', '')

In [8]:
# latitude and longitude information
# ==================================

# latitude of the states
lat = {'Delhi':28.7041, 'Haryana':29.0588, 'Kerala':10.8505, 'Rajasthan':27.0238,
       'Telengana':18.1124, 'Uttar Pradesh':26.8467, 'Ladakh':34.2996, 'Tamil Nadu':11.1271,
       'Jammu and Kashmir':33.7782, 'Punjab':31.1471, 'Karnataka':15.3173, 'Maharashtra':19.7515,
       'Andhra Pradesh':15.9129, 'Odisha':20.9517, 'Uttarakhand':30.0668, 'West Bengal':22.9868, 
       'Puducherry': 11.9416, 'Chandigarh': 30.7333, 'Chhattisgarh':21.2787, 'Gujarat': 22.2587, 
       'Himachal Pradesh': 31.1048, 'Madhya Pradesh': 22.9734, 'Bihar': 25.0961, 'Manipur':24.6637, 
       'Mizoram':23.1645, 'Goa': 15.2993, 'Andaman and Nicobar Islands': 11.7401, 'Assam' : 26.2006, 
       'Jharkhand': 23.6102, 'Arunachal Pradesh': 28.2180, 'Tripura': 23.9408, 'Nagaland': 26.1584, 
       'Meghalaya' : 25.4670, 'Dadar Nagar Haveli' : 20.1809}

# longitude of the states
long = {'Delhi':77.1025, 'Haryana':76.0856, 'Kerala':76.2711, 'Rajasthan':74.2179,
        'Telengana':79.0193, 'Uttar Pradesh':80.9462, 'Ladakh':78.2932, 'Tamil Nadu':78.6569,
        'Jammu and Kashmir':76.5762, 'Punjab':75.3412, 'Karnataka':75.7139, 'Maharashtra':75.7139,
        'Andhra Pradesh':79.7400, 'Odisha':85.0985, 'Uttarakhand':79.0193, 'West Bengal':87.8550, 
        'Puducherry': 79.8083, 'Chandigarh': 76.7794, 'Chhattisgarh':81.8661, 'Gujarat': 71.1924, 
        'Himachal Pradesh': 77.1734, 'Madhya Pradesh': 78.6569, 'Bihar': 85.3131, 'Manipur':93.9063, 
        'Mizoram':92.9376, 'Goa': 74.1240, 'Andaman and Nicobar Islands': 92.6586, 'Assam' : 92.9376, 
        'Jharkhand': 85.2799, 'Arunachal Pradesh': 94.7278, 'Tripura': 91.9882, 'Nagaland': 94.5624,
        'Meghalaya' : 91.3662, 'Dadar Nagar Haveli' : 73.0169}

# add latitude column based on 'Name of State / UT' column
df_bs['Latitude'] = df_bs['Name of State / UT'].map(lat)

# add longitude column based on 'Name of State / UT' column
df_bs['Longitude'] = df_bs['Name of State / UT'].map(long)

# df_bs.head(36)

In [9]:
# unique state names
df_bs['Name of State / UT'].unique()

array(['Andaman and Nicobar Islands', 'Andhra Pradesh',
       'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh',
       'Chhattisgarh', 'Dadar Nagar Haveli', 'Delhi', 'Goa', 'Gujarat',
       'Haryana', 'Himachal Pradesh', 'Jammu and Kashmir', 'Jharkhand',
       'Karnataka', 'Kerala', 'Ladakh', 'Madhya Pradesh', 'Maharashtra',
       'Manipur', 'Meghalaya', 'Mizoram', 'Odisha', 'Puducherry',
       'Punjab', 'Rajasthan', 'Tamil Nadu', 'Telengana', 'Tripura',
       'Uttarakhand', 'Uttar Pradesh', 'West Bengal'], dtype=object)

In [10]:
# number of missing values 
df_bs.isna().sum()

Name of State / UT           0
Total Confirmed cases*       0
Cured/Discharged/Migrated    0
Deaths**                     0
Date                         0
Latitude                     0
Longitude                    0
dtype: int64

In [11]:
# number of unique values 
df_bs.nunique()

Name of State / UT           33
Total Confirmed cases*       30
Cured/Discharged/Migrated    31
Deaths**                     20
Date                          1
Latitude                     33
Longitude                    29
dtype: int64

# Saving data

In [12]:
# saving data
# ===========

# file names as year-month-day.csv format
file_name = now.strftime("%Y_%m_%d")+'.csv'

# location for saving the file
file_loc = 'C:\\Users\\imdevskp\\Documents\\github\\covid_india\\.day_by_day_data\\'

# save file as a scv file
df_bs.to_csv(file_loc + file_name, index=False)

df_bs.head(36)

,Name of State / UT,Total Confirmed cases*,Cured/Discharged/Migrated,Deaths**,Date,Latitude,Longitude
0,Andaman and Nicobar Islands,33,33,0,2020-05-17,11.7401,92.6586
1,Andhra Pradesh,2355,1353,49,2020-05-17,15.9129,79.7400
2,Arunachal Pradesh,1,1,0,2020-05-17,28.2180,94.7278
3,Assam,92,41,2,2020-05-17,26.2006,92.9376
4,Bihar,1179,453,7,2020-05-17,25.0961,85.3131
5,Chandigarh,191,51,3,2020-05-17,30.7333,76.7794
6,Chhattisgarh,67,56,0,2020-05-17,21.2787,81.8661
7,Dadar Nagar Haveli,1,0,0,2020-05-17,20.1809,73.0169
8,Delhi,9333,3926,129,2020-05-17,28.7041,77.1025
9,Goa,17,7,0,2020-05-17,15.2993,74.1240


In [13]:
# column names 
# df_bs.columns

# Combining data

In [14]:
# list of all files available
# ! ls C:\Users\imdevskp\Documents\github\covid_india\.day_by_day_data

In [15]:
# location of the file
loc = "C:\\Users\\imdevskp\\Documents\\github\\covid_india\\.day_by_day_data\\"

# list of all files
files = glob.glob(loc+'2020*.csv')
   
# container for each day's data's dataframe
dfs = []

# loop through the files and append to the dfs list
for i in files:
    # read data
    df_temp = pd.read_csv(i)
    
    # rename columns
    df_temp = df_temp.rename(columns={'Cured':'Cured/Discharged'})
    df_temp = df_temp.rename(columns={'Cured/Discharged':'Cured/Discharged/Migrated', 
                                      'Total Confirmed cases *': 'Total Confirmed cases', 
                                      'Total Confirmed cases ': 'Total Confirmed cases', 
                                      'Total Confirmed cases* ': 'Total Confirmed cases'})
    df_temp = df_temp.rename(columns=lambda x: re.sub('Total Confirmed cases \(Including .* foreign Nationals\) ',
                                                      'Total Confirmed cases',x))
    df_temp = df_temp.rename(columns={'Deaths ( more than 70% cases due to comorbidities )':'Death', 
                                      'Deaths**':'Death'})
    
    # append to the df_s
    dfs.append(df_temp)
    
# print(dfs)

# concat dataframes
complete_data = pd.concat(dfs, ignore_index=True).sort_values(['Date'], ascending=True).reset_index(drop=True)

In [16]:
# complete_data.columns

## Preprocessing

In [17]:
# fix datatype
complete_data['Date'] = pd.to_datetime(complete_data['Date'])

# sort rows
complete_data = complete_data.sort_values(['Date', 'Name of State / UT']).reset_index(drop=True)

# fill missing values with 0
cols = ['Total Confirmed cases (Indian National)', 'Total Confirmed cases ( Foreign National )', 
              'Cured/Discharged/Migrated', 'Death']
complete_data[cols] = complete_data[cols].fillna(0).astype('int')

In [18]:
# rename state/UT names
complete_data['Name of State / UT'].replace('Chattisgarh', 'Chhattisgarh', inplace=True)
complete_data['Name of State / UT'].replace('Pondicherry', 'Puducherry', inplace=True) 

In [19]:
# select only rows with more than 1 case
complete_data = complete_data[complete_data['Total Confirmed cases']>0]

## Final dataframe

In [20]:
# random rows
complete_data.sample(3)

,Date,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Latitude,Longitude,Death,Total Confirmed cases
1729,2020-05-08,West Bengal,0,0,364,22.9868,87.8550,151,1548
874,2020-04-12,Ladakh,0,0,10,34.2996,78.2932,0,15
1098,2020-04-19,Karnataka,0,0,104,15.3173,75.7139,14,384


In [21]:
# complete data info
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2019 entries, 0 to 2026
Data columns (total 9 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   Date                                        2019 non-null   datetime64[ns]
 1   Name of State / UT                          2019 non-null   object        
 2   Total Confirmed cases (Indian National)     2019 non-null   int32         
 3   Total Confirmed cases ( Foreign National )  2019 non-null   int32         
 4   Cured/Discharged/Migrated                   2019 non-null   int32         
 5   Latitude                                    2019 non-null   float64       
 6   Longitude                                   2019 non-null   float64       
 7   Death                                       2019 non-null   int32         
 8   Total Confirmed cases                       2019 non-null   int64         
dtypes: dateti

## Save as .csv file

In [22]:
# save data in a csv file
complete_data.to_csv('complete.csv', index=False)